In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.linear_model import SGDRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error

In [3]:
!pip install codecarbon

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.9/502.9 kB 13.4 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 55.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.4/386.4 kB 19.0 MB/s eta 0:00:00
  Attempting uninstall: prompt_toolkit
    Found existing installation: prompt_toolkit 3.0.47
    Uninstalling prompt_toolkit-3.0.47:
      Successfully uninstalled prompt_toolkit-3.0.47
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 8.21.0 requires prompt-toolkit<3.1.0,>=3.0.41, but you have prompt-toolkit 3.0.36 which is incompatible.


In [4]:
from codecarbon import EmissionsTracker
tracker = EmissionsTracker()
tracker.start()

[codecarbon INFO @ 09:00:44] [setup] RAM Tracking...
[codecarbon INFO @ 09:00:44] [setup] GPU Tracking...
[codecarbon INFO @ 09:00:44] No GPU found.
[codecarbon INFO @ 09:00:44] [setup] CPU Tracking...
[codecarbon WARNING @ 09:00:44] No CPU tracking mode found. Falling back on CPU constant mode. 
 Linux OS detected: Please ensure RAPL files exist at \sys\class\powercap\intel-rapl to measure CPU

[codecarbon WARNING @ 09:00:45] We saw that you have a Intel(R) Xeon(R) CPU @ 2.20GHz but we don't know it. Please contact us.
[codecarbon INFO @ 09:00:45] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.20GHz
[codecarbon INFO @ 09:00:45] >>> Tracker's metadata:
[codecarbon INFO @ 09:00:45]   Platform system: Linux-5.15.154+-x86_64-with-glibc2.31
[codecarbon INFO @ 09:00:45]   Python version: 3.10.14
[codecarbon INFO @ 09:00:45]   CodeCarbon version: 2.7.1
[codecarbon INFO @ 09:00:45]   Available RAM : 31.358 GB
[codecarbon INFO @ 09:00:45]   CPU count: 4
[codecarbon INFO @ 09:

In [5]:
file_aforos = "/kaggle/input/pnav-prediccion-de-caudales/predictors_aforos.csv"
file_targets = "/kaggle/input/pnav-prediccion-de-caudales/target_emplazamientos_train.csv"
file_gfs = "/kaggle/input/pnav-prediccion-de-caudales/gfs.csv"

In [6]:
df_gfs = pd.read_csv(file_gfs)

[codecarbon INFO @ 09:01:03] Energy consumed for RAM : 0.000049 kWh. RAM Power : 11.759090423583986 W
[codecarbon INFO @ 09:01:03] Energy consumed for all CPUs : 0.000177 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:01:03] 0.000226 kWh of electricity used since the beginning.
[codecarbon INFO @ 09:01:18] Energy consumed for RAM : 0.000098 kWh. RAM Power : 11.759090423583986 W
[codecarbon INFO @ 09:01:18] Energy consumed for all CPUs : 0.000354 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:01:18] 0.000452 kWh of electricity used since the beginning.
[codecarbon INFO @ 09:01:33] Energy consumed for RAM : 0.000147 kWh. RAM Power : 11.759090423583986 W
[codecarbon INFO @ 09:01:33] Energy consumed for all CPUs : 0.000531 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:01:33] 0.000678 kWh of electricity used since the beginning.
[codecarbon INFO @ 09:01:48] Energy consumed for RAM : 0.000196 kWh. RAM Power : 11.759090423583986 W
[codecarbon INFO @ 09:01:48] Energy consumed f

In [7]:
df_aforos = pd.read_csv(file_aforos).drop(columns=["latitude", "longitude", "codigo"])
df_aforos["time"] = pd.to_datetime(df_aforos["time"])

# time es el target, tiempo a 48 horas vista
df_targets = pd.read_csv(file_targets)
df_loc_targets = df_targets[["name", "latitude", "longitude"]].drop_duplicates()
df_targets = df_targets.drop(columns=["latitude", "longitude", "codigo"])
df_targets["time"] = pd.to_datetime(df_targets["time"])

In [8]:
df_gfs = pd.read_csv(file_gfs)

[codecarbon INFO @ 09:04:03] Energy consumed for RAM : 0.000637 kWh. RAM Power : 11.759090423583986 W
[codecarbon INFO @ 09:04:03] Energy consumed for all CPUs : 0.002302 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:04:03] 0.002939 kWh of electricity used since the beginning.
[codecarbon INFO @ 09:04:18] Energy consumed for RAM : 0.000686 kWh. RAM Power : 11.759090423583986 W
[codecarbon INFO @ 09:04:18] Energy consumed for all CPUs : 0.002479 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:04:18] 0.003165 kWh of electricity used since the beginning.
[codecarbon INFO @ 09:04:33] Energy consumed for RAM : 0.000735 kWh. RAM Power : 11.759090423583986 W
[codecarbon INFO @ 09:04:33] Energy consumed for all CPUs : 0.002656 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:04:33] 0.003391 kWh of electricity used since the beginning.
[codecarbon INFO @ 09:04:48] Energy consumed for RAM : 0.000784 kWh. RAM Power : 11.759090423583986 W
[codecarbon INFO @ 09:04:48] Energy consumed f

In [9]:
def haversine_np(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)
    
    All args must be of equal length.    
    
    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
    
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    
    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2
    
    c = 2 * np.arcsin(np.sqrt(a))
    km = 6378.137 * c
    return km

In [10]:
for _, row in df_loc_targets.iterrows():
    name_emplazamiento = row["name"].replace(" ", "_")
    latitud_emplazamiento = np.array([row["latitude"]] * len(df_gfs))
    longitud_emplazamiento = np.array([row["longitude"]] * len(df_gfs))
    dist = haversine_np(df_gfs["longitude"], df_gfs["latitude"],
                        longitud_emplazamiento, latitud_emplazamiento)
    df_gfs[f"dist_{name_emplazamiento}"] = dist

df_gfs["min_dist"] = df_gfs.filter(like="dist").min(axis=1)

[codecarbon INFO @ 09:06:35] Energy consumed for RAM : 0.001134 kWh. RAM Power : 11.759090423583986 W
[codecarbon INFO @ 09:06:35] Energy consumed for all CPUs : 0.004098 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:06:35] 0.005232 kWh of electricity used since the beginning.
[codecarbon INFO @ 09:06:50] Energy consumed for RAM : 0.001183 kWh. RAM Power : 11.759090423583986 W
[codecarbon INFO @ 09:06:50] Energy consumed for all CPUs : 0.004276 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:06:50] 0.005458 kWh of electricity used since the beginning.
[codecarbon INFO @ 09:06:50] 0.002091 g.CO2eq/s mean an estimation of 65.93626970920297 kg.CO2eq/year


In [11]:
class GFSFeatures:
    def __init__(self, df_gfs_in: pd.DataFrame, min_dist: float) -> None:
        self._df_gfs = df_gfs_in
        self.min_dist = min_dist

    def get(self) -> pd.DataFrame:
        df_out = None
        for emplazamiento in range(5):
            df_gfs_i = self._df_gfs[self._df_gfs[f"dist_Emplazamiento_{emplazamiento}"] <= self.min_dist]

            rows = []
            idxs = []
            for time, grp in df_gfs.drop(columns="valid_time").groupby("time"):
                dists = grp[f"dist_Emplazamiento_{emplazamiento}"]
                grp_i = grp.drop(columns=["time", "latitude",	"longitude", "dist_Emplazamiento_0",
                                          "dist_Emplazamiento_1", "dist_Emplazamiento_2",
                                          "dist_Emplazamiento_3", "dist_Emplazamiento_4"])
                grp_i = grp_i.fillna(grp_i.median())
                idxs.append(time)
                rows.append(
                    np.average(grp_i.values, weights=1.0/dists, axis=0)
                )
            df_emp = pd.DataFrame(rows, index=pd.to_datetime(idxs), columns=[f"{x}__emp_{emplazamiento}" for x in grp_i.columns])

            if df_out is None:
                df_out = df_emp
            else:
                df_out = df_out.join(df_emp, how="inner")
        return df_out

In [12]:
df_gfs_features = GFSFeatures(df_gfs, min_dist=20).get()

[codecarbon INFO @ 09:07:05] Energy consumed for RAM : 0.001232 kWh. RAM Power : 11.759090423583986 W
[codecarbon INFO @ 09:07:05] Energy consumed for all CPUs : 0.004453 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:07:05] 0.005684 kWh of electricity used since the beginning.
[codecarbon INFO @ 09:07:20] Energy consumed for RAM : 0.001281 kWh. RAM Power : 11.759090423583986 W
[codecarbon INFO @ 09:07:20] Energy consumed for all CPUs : 0.004630 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:07:20] 0.005910 kWh of electricity used since the beginning.
[codecarbon INFO @ 09:07:35] Energy consumed for RAM : 0.001330 kWh. RAM Power : 11.759090423583986 W
[codecarbon INFO @ 09:07:35] Energy consumed for all CPUs : 0.004807 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:07:35] 0.006136 kWh of electricity used since the beginning.
[codecarbon INFO @ 09:07:50] Energy consumed for RAM : 0.001379 kWh. RAM Power : 11.759090423583986 W
[codecarbon INFO @ 09:07:50] Energy consumed f

In [13]:
df_targets_24 = df_targets.copy()
df_targets_24["time"] += pd.Timedelta(days=1)
df_targets_48 = df_targets.copy()
df_targets_48["time"] += pd.Timedelta(days=2)

In [15]:
targets_24 = df_targets_24.pivot(index="time", columns="name", values="caudal_m3/s")
targets_24.columns = [x.replace(" ", "_") + "_24h" for x in targets_24.columns]
targets_48 = df_targets_48.pivot(index="time", columns="name", values="caudal_m3/s")
targets_48.columns = [x.replace(" ", "_") + "_48h" for x in targets_48.columns]

In [16]:
all_targets = targets_24.join(targets_48, how="inner")

In [17]:
target_cols = all_targets.columns

In [18]:
test_dates = pd.DatetimeIndex(['2023-04-12', '2023-04-13', '2023-05-03', '2023-05-04',
               '2023-06-08', '2023-06-09', '2023-07-05', '2023-07-06',
               '2023-07-31', '2023-08-01', '2023-09-05', '2023-09-06',
               '2023-09-26', '2023-09-27', '2023-11-03', '2023-11-04',
               '2023-12-03', '2023-12-04', '2023-12-25', '2023-12-26',
               '2024-01-18', '2024-01-19', '2024-02-12', '2024-02-13',
               '2024-03-11', '2024-03-12'],
              dtype='datetime64[ns]')

In [19]:
# 1.- Caudal en cada aforo (actual, hace 24 horas, e incrementos [relativo y absoluto])
# 5 caudales * 4 variables = 20 variables

In [20]:
def create_features_from_measurements(df_input: pd.DataFrame) -> pd.DataFrame:
    df_input_24h_ago = pd.DataFrame(df_input)
    df_input_24h_ago["time"] = df_input_24h_ago["time"] + pd.Timedelta(days=1)
    df_input_24h_ago.columns = ["time", "caudal_m3/s_hace24", "name"]
    df_merged = df_input.merge(df_input_24h_ago, how="left")
    empty_24h_ago = df_merged["caudal_m3/s_hace24"].isna()
    df_merged.loc[empty_24h_ago, "caudal_m3/s_hace24"] = df_merged[empty_24h_ago]["caudal_m3/s"]

    df_merged["inc_abs_caudal_24"] = df_merged["caudal_m3/s"] - df_merged["caudal_m3/s_hace24"]
    df_merged["inc_rel_caudal_24"] = ((df_merged["caudal_m3/s"] - df_merged["caudal_m3/s_hace24"]) /\
                                                 (1 + df_merged["caudal_m3/s_hace24"])).fillna(0)
    df_features = None
    for current_name, grp in df_merged.groupby("name"):
        grp = grp.drop(columns=["name"]).set_index("time")
        current_name = current_name.replace(" ", "_")
        grp.columns = [f"{x}__{current_name}" for x in grp.columns]
        if df_features is None:
            df_features = grp
        else:
            df_features = df_features.join(grp)
    return df_features

In [21]:
df_features_aforos = create_features_from_measurements(df_aforos)

In [22]:
df_features_targets = create_features_from_measurements(df_targets)

In [23]:
df_feats = df_features_aforos.sort_index().join(df_features_targets, how="inner").join(df_gfs_features, how="inner").ffill().bfill()

In [24]:
df_all = df_feats.join(all_targets, how="inner")

In [25]:
df_test_set = df_feats[~df_feats.index.isin(df_all.index)]

In [26]:
df_train_set = df_all.loc[[x for x in df_all.index if x not in test_dates]]

In [27]:
N = len(df_train_set)
Ntrain = int(0.80*N)
Ntest = N - Ntrain

In [28]:
df_train_train_set = df_train_set.head(Ntrain)
df_train_test_set = df_train_set.tail(Ntest)

In [29]:
assert len(df_train_test_set) + len(df_train_train_set) == N


In [30]:
assert not set(df_train_train_set.index) & set(df_train_test_set.index)

In [31]:
all_scores = {}
param_grid = {
    'pca__n_components': [2, 3, 5, 7, 10, 15, 20, 25],
    'regressor__alpha': [0.0001, 0.001, 0.1, 0.5],    
}

Xpred = df_test_set.sort_index()
predictions = {}
best_estimators = {}
n_components = 3
alpha = 0.0001

for tg_var in target_cols:
    print(f"Optimizando para target {tg_var}")
    ytrain = df_train_train_set[tg_var]
    Xtrain = df_train_train_set.drop(columns=target_cols)

    ts = TimeSeriesSplit(n_splits=2)
    
    ytest = df_train_test_set[tg_var]
    Xtest = df_train_test_set.drop(columns=target_cols)

    clf = Pipeline([('var_thres', VarianceThreshold()),
                    ('pca', PCA(n_components=n_components)),
                    ('scaler', StandardScaler()),
                    ('regressor', SGDRegressor(epsilon=0, loss="epsilon_insensitive", random_state=42, alpha=alpha, max_iter=2000))]
    )

    grid_search = GridSearchCV(clf, param_grid=param_grid, scoring="neg_mean_absolute_error")
    grid_search.fit(Xtrain, ytrain)
    best_parameters = grid_search.best_params_
    best_estimator = grid_search.best_estimator_
    best_estimators[tg_var] = best_estimator

    # estimation of predictive power
    yhat = best_estimator.predict(Xtest)
    score_i = mean_absolute_error(ytest, yhat)
    all_scores[tg_var] = score_i

    # classify test set for this model
    ytest_hat = best_estimator.predict(Xpred)
    predictions[tg_var] = ytest_hat

Optimizando para target Emplazamiento_0_24h


[codecarbon INFO @ 09:09:51] Energy consumed for RAM : 0.001772 kWh. RAM Power : 11.759090423583986 W
[codecarbon INFO @ 09:09:51] Energy consumed for all CPUs : 0.006408 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:09:51] 0.008180 kWh of electricity used since the beginning.
[codecarbon INFO @ 09:10:06] Energy consumed for RAM : 0.001821 kWh. RAM Power : 11.759090423583986 W
[codecarbon INFO @ 09:10:06] Energy consumed for all CPUs : 0.006585 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:10:06] 0.008406 kWh of electricity used since the beginning.


Optimizando para target Emplazamiento_1_24h


[codecarbon INFO @ 09:10:21] Energy consumed for RAM : 0.001870 kWh. RAM Power : 11.759090423583986 W
[codecarbon INFO @ 09:10:21] Energy consumed for all CPUs : 0.006762 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:10:21] 0.008633 kWh of electricity used since the beginning.


Optimizando para target Emplazamiento_2_24h


...

[codecarbon INFO @ 09:12:51] Energy consumed for RAM : 0.002361 kWh. RAM Power : 11.759090423583986 W
[codecarbon INFO @ 09:12:51] Energy consumed for all CPUs : 0.008534 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:12:51] 0.010894 kWh of electricity used since the beginning.
[codecarbon INFO @ 09:12:51] 0.002091 g.CO2eq/s mean an estimation of 65.92751736522156 kg.CO2eq/year
[codecarbon INFO @ 09:13:06] Energy consumed for RAM : 0.002409 kWh. RAM Power : 11.759090423583986 W
[codecarbon INFO @ 09:13:06] Energy consumed for all CPUs : 0.008711 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:13:06] 0.011120 kWh of electricity used since the beginning.


Optimizando para target Emplazamiento_3_48h


[codecarbon INFO @ 09:13:21] Energy consumed for RAM : 0.002459 kWh. RAM Power : 11.759090423583986 W
[codecarbon INFO @ 09:13:21] Energy consumed for all CPUs : 0.008888 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:13:21] 0.011347 kWh of electricity used since the beginning.
[codecarbon INFO @ 09:13:36] Energy consumed for RAM : 0.002508 kWh. RAM Power : 11.759090423583986 W
[codecarbon INFO @ 09:13:36] Energy consumed for all CPUs : 0.009065 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:13:36] 0.011573 kWh of electricity used since the beginning.


Optimizando para target Emplazamiento_4_48h


[codecarbon INFO @ 09:13:51] Energy consumed for RAM : 0.002557 kWh. RAM Power : 11.759090423583986 W
[codecarbon INFO @ 09:13:51] Energy consumed for all CPUs : 0.009242 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:13:51] 0.011799 kWh of electricity used since the beginning.
[codecarbon INFO @ 09:14:06] Energy consumed for RAM : 0.002605 kWh. RAM Power : 11.759090423583986 W
[codecarbon INFO @ 09:14:06] Energy consumed for all CPUs : 0.009419 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:14:06] 0.012025 kWh of electricity used since the beginning.


In [32]:
test_dates2 = df_test_set.index

In [33]:
test_dates2 = df_test_set.index + pd.Timedelta(days=-2)

In [34]:
df_submission = pd.DataFrame(index=test_dates2, data=predictions)

In [35]:
out_rows = []
df_sub = df_submission.filter(like="48h")
for idx, row in df_sub.iterrows():
    for col in df_sub.columns:
        result_row = [str(idx)[:10] + "_" + col.replace("_48h", "").replace("_", " "), row[col]] 
        out_rows.append(result_row)

df_submission = pd.DataFrame(out_rows, columns=["time_name", "caudal_m3/s"]).set_index("time_name")

In [36]:
df_submission.to_csv("submission.csv")

In [37]:
tracker.stop()

[codecarbon INFO @ 09:14:10] Energy consumed for RAM : 0.002617 kWh. RAM Power : 11.759090423583986 W
[codecarbon INFO @ 09:14:10] Energy consumed for all CPUs : 0.009460 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:14:10] 0.012077 kWh of electricity used since the beginning.
/opt/conda/lib/python3.10/site-packages/codecarbon/output_methods/file.py:52: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame.from_records([dict(total.values)])])


0.0016756591507265538